# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

Clase 10-11-20 1:00 aprox
Todos los graficos con titulos, escrbir bien etiquetas, usar bien colores(no como matlab), si pintas dos datos con el mismo color usar mapas
de colores con mas colores

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
#alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)
confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.tail(7)

,grupo_de_edad,fecha,fallecidos
1652,0-39,2020-12-01,350
1653,40-49,2020-12-01,519
1654,50-59,2020-12-01,1547
1655,60-69,2020-12-01,3256
1656,70-79,2020-12-01,4337
1657,80-89,2020-12-01,3987
1658,>=90,2020-12-01,1434


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [7]:
alt.Chart(fallecidos_etareo).mark_bar().encode(  
    x=alt.X('grupo_de_edad', axis=alt.Axis(title='Grupo Etario')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos')),
    color='grupo_de_edad:N',
    tooltip=['grupo_de_edad','fallecidos'],
    ).properties(
    width=600,
    height=500,
    title="FALLECIDOS POR GRUPO ETARIO A LA FECHA"
)

alt.Chart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [8]:
alt.Chart(activos).mark_bar().encode(
    alt.X("comuna:O"),
    y='poblacion',
    color=alt.Color('poblacion:Q'),
    tooltip=['comuna','poblacion'],
   
)

alt.Chart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [9]:
fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


In [10]:
# 1:07:00
#Serie temporal (funcion del tiempo)
#Quiero ver si en un dia en particular hay mas fallecimientos del grupo etario de los 20 a los 30 por decir
# o de los 50 a los 60 pero ese mismo dia puedo decir que hubo una cantidad de fallecidos totales de X
#tener ojo con el tipo de grafico

# FIX ME #  Ver si en un dia en particular hay mas fallecidos con una edad, ver tipo de grafico

f_etarios=alt.Chart(fallecidos_etareo).mark_line().encode(   # Probar con mark_line y con esto algo se entiende pero no bien
    x='fecha',  # fecha
    y='fallecidos',  # Precio de accion
    color=alt.Color('grupo_de_edad:N',legend=alt.Legend(title="Grupo Etareo")),  # se puede separar por distintas compañias con el canal de color, sin el color  cada 
    tooltip=["grupo_de_edad","fecha","fallecidos"]
    #registro va a ser un punto y va a tratar de unir todos estos puntos 
).properties(
    width=600, height=400
).interactive()

f_totales = alt.Chart(fallecidos_etareo).mark_line().encode(    #aca igual se puede usar el base points = base.mark_point().encode(
    x='fecha',
    y='sum(fallecidos)',
    color=alt.value('blue'),
    #color=alt.Color('sum(fallecidos):Q',legend=alt.Legend(title="Fallecidos Totales")),
    #color=alt.Color("sum(fallecidos):Q",scale=alt.Scale(domain=['sum(fallecidos)'], range=['blue']),legend=alt.Legend(title="Fallecidos Totales")),
    tooltip=["fecha","sum(fallecidos)"]
).properties(
    width=600, height=400
).interactive()

f_etarios + f_totales

alt.LayerChart(...)

**Comentarios:** RESPONDE AQUÍ

In [11]:
#para series de tiempo entrega una linea para ver puntps en el grafico

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

# The basic line
line1 = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(
    x='fecha',
    y='fallecidos',
    color='grupo_de_edad',
    tooltip=["grupo_de_edad","fecha","fallecidos"]
)

line2 = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(
    x='fecha',
    y='sum(fallecidos)',
    #color='sum(fallecidos)'
    #tooltip=["grupo_de_edad","fecha","fallecidos"]
)
line_f=line1 + line2

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(fallecidos_etareo).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text1 = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'fallecidos', alt.value(' '))
)


text2 = line.mark_text(align='left', dx=5, dy=-5).encode(
   text=alt.condition(nearest, 'sum(fallecidos)', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(fallecidos_etareo).mark_rule(color='gray').encode(
   x='fecha',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line_f, selectors, points, rules,  text1, text2
).properties(
    width=600, height=300
)



NameError: name 'line' is not defined

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
#Dos graficos iguales y uno muy distinto
#brush = alt.selection(type='interval')

line1 = alt.Chart(confirmados).mark_line().encode(
    x='fecha',
    y='sum(casos_confirmados)',
    #color=alt.condition(brush, 'sum(casos_confirmados)', alt.value('lightgray'))
)#.add_selection(
   # brush
#)

line2 = alt.Chart(fallecidos).mark_line().encode(
    x='fecha',
    y='sum(fallecidos)',
    #color='sum(fallecidos)'   
)

line3 = alt.Chart(activos).mark_line().encode(
    x='fecha',
    y='sum(casos_activos)',
    #color='sum(casos_activos)'
    
#).transform_filter(
 #   brush
)

line1 | line2 | line3

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [12]:
# FIX ME #  Va a quedar feo, decir pq esta feo
#Ver si esta correcta la tasa de incidencia

alt.Chart(confirmados).mark_line().encode(
    x='fecha',
    y='sum(tasa)',
    color='region',
    #color=alt.condition(brush, 'sum(casos_confirmados)', alt.value('lightgray'))
)

alt.Chart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [13]:
#Pensar en 2 scatterplot, ver si es util colocar tamañode circulo para ver tamaño poblacion
#Poner por ejemplo: si es util colocar el tamaño del circulo ya que es util para diferenciar los tamaños
#Colocar en funcion de tamaño de punto y revisar si tasa es correcta
fecha1=confirmados.query( "fecha == '2020-04-13'" )

casos=alt.Chart(fecha1).mark_circle().encode(
    x='comuna',
    y='casos_confirmados',
    #opacity=alt.value(0.3),
    tooltip=['region','comuna','casos_confirmados'],
    #size=alt.value(120),
    size='poblacion:Q',
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'))
    #color='region:N'
).properties(
    width=5000,
    height=600
).interactive()

tasa=alt.Chart(fecha1).mark_circle().encode(
    x='comuna',
    y='tasa',
    #opacity=alt.value(0.3),
    tooltip=['region','comuna','tasa'],
    #size=alt.value(120),
    size='poblacion:Q',
    color=alt.Color('region:N', scale=alt.Scale(scheme='tableau20'))
    #color='region:N'
).properties(
    width=2000,
    height=600
).interactive()

casos+tasa

alt.LayerChart(...)

In [14]:
fecha1['tasa'].max()

907.715582450832

In [15]:
fecha2=confirmados.query( "fecha == '2020-11-06'" )

casos=alt.Chart(fecha2).mark_point(size=60).encode(
    x='comuna',
    y='casos_confirmados',
    tooltip=['casos_confirmados'],
    size=alt.value(80),
    color='region:N'
    
).properties(
    width=5000,
    height=200
)

tasa=alt.Chart(fecha2).mark_circle(size=60).encode(
    x='comuna',
    y='tasa',
    tooltip=['tasa'],
    size=alt.value(80),
    color='region:N'
).properties(
    width=5000,
    height=200
)

casos+tasa

alt.LayerChart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [16]:

line1 = alt.Chart(activos).mark_line().encode(
    x='fecha',
    y='casos_activos',
    color='comuna',
    tooltip=['comuna','casos_activos']
)

line1 

alt.Chart(...)

In [17]:
Arica_y_Parinacota=activos.query( "region == 'Arica y Parinacota'" )
r1 = alt.Chart(Arica_y_Parinacota).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Tarapaca=activos.query( "region == 'Tarapaca'" )
r2 = alt.Chart(Tarapaca).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Antofagasta=activos.query( "region == 'Antofagasta'" )
r3 = alt.Chart(Antofagasta).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Atacama=activos.query( "region == 'Atacama'" )
r4 = alt.Chart(Atacama).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Coquimbo=activos.query( "region == 'Coquimbo'" )
r5 = alt.Chart(Coquimbo).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Valparaiso=activos.query( "region == 'Valparaiso'" )
r6 = alt.Chart(Valparaiso).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Metropolitana=activos.query( "region == 'Metropolitana'" )
r7 = alt.Chart(Metropolitana).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Del_Libertador=activos.query( "region == 'Del Libertador General Bernardo O’Higgins'" )
r8 = alt.Chart(Del_Libertador).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Maule=activos.query( "region == 'Maule'" )
r9 = alt.Chart(Maule).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Nuble=activos.query( "region == 'Nuble'" )
r10 = alt.Chart(Nuble).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Biobio=activos.query( "region == 'Biobio'" )
r11 = alt.Chart(Biobio).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

La_Araucania=activos.query( "region == 'La Araucania'" )
r12 = alt.Chart(La_Araucania).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Los_Rios=activos.query( "region == 'Los Rios'" )
r13 = alt.Chart(Los_Rios).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Los_Lagos=activos.query( "region == 'Los Lagos'" )
r14 = alt.Chart(Los_Lagos).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Aysen=activos.query( "region == 'Aysen'" )
r15 = alt.Chart(Aysen).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

Magallanes_Antartica=activos.query( "region == 'Magallanes y la Antartica'" )
r16 = alt.Chart(Magallanes_Antartica).mark_line().encode(x='fecha',y='casos_activos',color='comuna',tooltip=['comuna','casos_activos'])

(r1 | r2 | r3) & (r4 | r5 | r6) & (r7 | r8 | r9) & (r10 | r11 | r12) & (r13 | r14 | r15) & r16

alt.VConcatChart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [18]:
region_names = {
    "Araucanía": "La Araucanía",
    "La Araucania": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Biobio": "Biobío",
    "Los Rios": "Los Ríos",
    "Nuble": "Ñuble",
    "Valparaiso": "Valparaíso",
    "Tarapaca": "Tarapacá"  
}

In [19]:
activos=activos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
activos_region = (
   activos.groupby(["region"])
    .agg(
        prom_activos=("casos_activos", "mean"),        
    )
)
activos_region

,prom_activos
region,
Antofagasta,78.112795
Arica y Parinacota,100.310606
Atacama,33.713805
Aysén del General Carlos Ibáñez del Campo,4.406061
Biobío,46.290174
Coquimbo,33.002020
La Araucanía,16.421402
Libertador General Bernardo O'Higgins,24.648760
Los Lagos,24.295455


In [20]:
confirmados=confirmados.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
max_confirm= (
   confirmados.groupby(["region"])
    .agg(
        max_conf=("casos_confirmados", "max"),
        #attack_max=("attack", "max")
    )
)
max_confirm

,max_conf
region,
Antofagasta,12831.0
Arica y Parinacota,10530.0
Atacama,4820.0
Aysén del General Carlos Ibáñez del Campo,1086.0
Biobío,6401.0
Coquimbo,5099.0
La Araucanía,5836.0
Libertador General Bernardo O'Higgins,9046.0
Los Lagos,7812.0


In [21]:
fallecidos=fallecidos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
max_fallec= (
   fallecidos.groupby(["region"])
    .agg(
        max_fallec=("fallecidos", "max"),
        )
).drop(['Total'],axis=0)

max_fallec

,max_fallec
region,
Antofagasta,562.0
Arica y Parinacota,207.0
Atacama,106.0
Aysén del General Carlos Ibáñez del Campo,14.0
Biobío,610.0
Coquimbo,270.0
La Araucanía,229.0
Libertador General Bernardo O'Higgins,544.0
Los Lagos,203.0


In [22]:
grafico=activos_region.merge(
       max_confirm,
       how="left",
       on="region"
    ).merge(
        max_fallec,
        how="left",
        on="region"
   ).reset_index()
grafico

,region,prom_activos,max_conf,max_fallec
0,Antofagasta,78.112795,12831.0,562.0
1,Arica y Parinacota,100.310606,10530.0,207.0
2,Atacama,33.713805,4820.0,106.0
3,Aysén del General Carlos Ibáñez del Campo,4.406061,1086.0,14.0
4,Biobío,46.290174,6401.0,610.0
5,Coquimbo,33.002020,5099.0,270.0
6,La Araucanía,16.421402,5836.0,229.0
7,Libertador General Bernardo O'Higgins,24.648760,9046.0,544.0
8,Los Lagos,24.295455,7812.0,203.0
9,Los Ríos,17.915404,2940.0,55.0


In [23]:

grafico1=alt.Chart(grafico).mark_circle(size=60).encode(
    x='region',
    y='prom_activos',
    tooltip=['region','prom_activos'],
    size=alt.value(50),
    color='region:N'
    #color='prom_activos:N'
).interactive()

grafico2=alt.Chart(grafico).mark_circle(size=50).encode(
    x='region',
    y='max_conf',
    tooltip=['region','max_conf'],
    size=alt.value(150),
    color='region:N'
    #color='max_conf:N'
).interactive()

grafico3=alt.Chart(grafico).mark_circle(size=40).encode(
    x='region',
    y='max_fallec',
    tooltip=['region','max_fallec'],
    size=alt.value(40),
    color='region:N'
    #color='max_fallec:N'
).interactive()

(grafico1+grafico2+grafico3).properties(
    width=1000,
    height=700
)

# falta tratar de dejar 3 colores para diferenciar puntos
#ver si sirve tamaño de puntos

alt.LayerChart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [24]:
import geopandas as gpd   # instalar geopandas, agregarlo en enviroment
from pathlib import Path


In [25]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [26]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True}, # mapea en el mapa bidimensional
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [27]:
confirmados1=confirmados.groupby(["region", "fecha"]).agg(
        poblacion=("poblacion", "sum"),
        codigo_region=("region_id", "mean"),
        casos_confirmados=("casos_confirmados", "sum"),
        ).reset_index()
confirmados2=confirmados.drop(['provincia_id','provincia','comuna_id','comuna'],axis=1).groupby(
        ["region", "fecha"]).apply(lambda df: df["casos_confirmados"].sum()/df["poblacion"].sum()*100000).reset_index().rename(columns={  0 : 'tasa'})
confirmados_f=confirmados1.merge(confirmados2,how='right', on=['region', 'fecha']).replace({'poblacion': 82295.0}, 107297.0)

In [28]:
activos=activos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
activos1=activos.groupby(["region", "fecha"]).agg(
        poblacion=("poblacion", "sum"),
        codigo_region=("codigo_region", "max"),
        casos_activos=("casos_activos", "sum"),
        ).reset_index()

In [29]:
conf_act=confirmados_f.merge(activos1,how='inner', on=['region', 'fecha','poblacion','codigo_region'])

In [30]:
fallecidos=fallecidos.assign(region=lambda x: x["region"].map(region_names).fillna(x["region"]))
fallec= (
   fallecidos.groupby(["region","fecha"])
    .agg(
        fallecidos=("fallecidos", "max"),
        )
).drop(['Total'],axis=0).reset_index()

In [31]:
conf_act_fall=conf_act.merge(fallec,how='inner', on=['region', 'fecha'])

In [32]:
region_names1 = {
    "Región Metropolitana de Santiago": "Metropolitana de Santiago",
    "Región de Antofagasta": "Antofagasta",
    "Región de Arica y Parinacota": "Arica y Parinacota",
    "Región de Atacama": "Atacama",
    "Región de Aysén del Gral.Ibañez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    "Región de Coquimbo": "Coquimbo",
    "Región de La Araucanía": "La Araucanía",
    "Región de Los Lagos": "Los Lagos",
    "Región de Los Ríos": "Los Ríos",
    "Región de Magallanes y Antártica Chilena": "Magallanes y de la Antártica Chilena",
    "Región de Tarapacá": "Tarapacá",
    "Región de Valparaíso": "Valparaíso",
    "Región de Ñuble": "Ñuble",
    "Región del Bío-Bío": "Biobío",
    "Región del Libertador Bernardo O'Higgins": "Libertador General Bernardo O'Higgins",
    "Región del Maule": "Maule"
}
    
regiones1=regiones.drop(['objectid','objectid','cir_sena','codregion','area_km','st_area_sh','st_length_'],axis=1).rename(
columns={'Region':'region'}).assign(region=lambda x: x["region"].map(region_names1).fillna(x["region"]))

In [33]:
casos_geo = (conf_act_fall.merge(regiones1,how='inner', on=['region'])).sort_values(["codigo_region","fecha"])

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [34]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))
casos_geo#.dtypes

,region,fecha,poblacion,codigo_region,casos_confirmados,tasa,casos_activos,fallecidos,geometry
858,Tarapacá,2020-04-13,382773.0,1.0,52.0,13.585075,30.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
859,Tarapacá,2020-04-15,382773.0,1.0,62.0,16.197590,35.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
860,Tarapacá,2020-04-17,382773.0,1.0,73.0,19.071356,38.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
861,Tarapacá,2020-04-20,382773.0,1.0,93.0,24.296385,42.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
862,Tarapacá,2020-04-24,382773.0,1.0,130.0,33.962688,61.0,0.0,"MULTIPOLYGON (((-7807680.075 -2295867.719, -78..."
...,...,...,...,...,...,...,...,...,...
1051,Ñuble,2020-11-13,511551.0,16.0,10562.0,2064.701271,390.0,162.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1052,Ñuble,2020-11-16,511551.0,16.0,10761.0,2103.602573,429.0,163.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1053,Ñuble,2020-11-20,511551.0,16.0,11022.0,2154.623879,476.0,168.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1054,Ñuble,2020-11-23,511551.0,16.0,11240.0,2197.239376,492.0,172.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."


### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [35]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x["fecha"] == fecha]#[# FIX ME #
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col# FIX ME #
        ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=250,
        height=600
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [36]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

alt.Chart(...)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [37]:
from ipywidgets import interactive, fixed
import ipywidgets as widgets
from ipywidgets import interactive, interact
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

In [38]:
col_widget = widgets.Dropdown(options=["poblacion","casos_confirmados","tasa","casos_activos","fallecidos"]
                              ,description='COLUMNA',continuous_update=False)                        

In [39]:
fecha_widget = widgets.DatePicker(description='FECHA',continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [40]:
covid_dashboard = interactive(covid_chile_chart,fecha=fecha_widget,col=col_widget)
covid_dashboard

interactive(children=(DatePicker(value=None, description='FECHA'), Dropdown(description='COLUMNA', options=('p…

**Comentarios:** RESPONDE AQUÍ